In [177]:
import pycbc.noise
import pycbc.psd
import pycbc.filter
from pycbc.types import timeseries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import signal
import matplotlib.pyplot as plt
from numpy.random import uniform, randint
import pylab
from pycbc.filter import sigma
from pycbc.waveform import get_td_waveform
from pycbc.psd import interpolate
from pycbc.types.timeseries import load_timeseries

#import false signal functions
from ipynb.fs.full.falsig_functions import random_false_sig, flip_gw, gaussPulse_2B, expsin_2B, sawtooth_2B, chirp_2B

ImportError: cannot import name 'gaussPulse_2B'

In [164]:
# Generate 1 seconds of noise at 4096 Hz, randomized
# argument:
# sig: timeseries
# snr: desired signal-noise ratio
# output noise is scaled to desired snr

def noise(sig,snr):
    # psd
    flow = 20.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    
    # un-scaled noise
    delta_t = 1.0 / 4096
    tsamples = int(1.0 / delta_t)
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)
    
    # Scale
    new_psd = pycbc.psd.estimate.interpolate(psd, sig.delta_f)  # First interpolate psd to desire delta_f
    current_snr = sigma(sig, psd=new_psd, low_frequency_cutoff=20.0)
    c = current_snr/snr
    scaled_ts = c * ts

    return scaled_ts

In [165]:
# Generate gravitational waveform with specified mass parameters.
# Other arguments fixed
# return the plus polarization

# For the ease of merging, the waveform is shifted and its start_time is reset to 0.

def gw(m1,m2):
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=m1,    
                          mass2=m2,
                          delta_t=1.0/4096,
                          f_lower=20,
                          distance=100)
    # resize to window of 1 second
    hp.resize(4096)
    
    # randomly shift
    # To fit the whole waveform in the 1 second window, need to specify a cyclic shift range. 
    # 0.05 act as buffer since signal still oscillates a little after merging.
    shift_range = 1 - abs(hp.start_time)-0.05 
    shifted = hp.cyclic_time_shift(uniform(0.0,shift_range))

    # reset start time
    shifted.start_time = 0
    
    return shifted
    

In [166]:
def burried_gw(m1,m2,snr):
    sig = gw(m1,m2)
    noi = noise(sig,snr)
    
    return noi + sig

In [167]:
def burried_false(mag,snr):
    fal = random_false_sig(mag)
    fal.resize(4096)
    noi = noise(fal,snr)
    
    return noi + fal

In [168]:
def burried_flip(m1,m2,snr):
    hp = gw(m1,m2)
    flipped = flip_gw(hp)
    noi = noise(hp,snr)
    
    return noi + flipped

In [169]:
# scaling factor. Originally noise contained within around 1*10^-21
def empty_gaussian(c):
    # psd
    flow = 20.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    
    # un-scaled noise
    delta_t = 1.0 / 4096
    tsamples = int(1.0 / delta_t)
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)
    
    # Scale
    scaled_ts = c * ts

    return scaled_ts

In [170]:
# A crude visualization because not sure how to tune parameters for qtransform.
# Show the plot

def visualize(wave):
    times, freqs, power = wave.qtransform(.001, logfsteps=100,
                                                qrange=(4, 4),
                                                frange=(20, 512),)
    pylab.figure(figsize=[15, 3])
    pylab.pcolormesh(times, freqs, power**0.5)
    pylab.yscale('log')
    pylab.show()

In [171]:
'''
# test
test1 = burried_gw(30,35,1000)
test2 = burried_flip(30,35,1000)
test3 = burried_false(-21,1000)

# plot
pylab.plot(test1.sample_times, test1)
pylab.show()
# plot
pylab.plot(test2.sample_times, test2)
pylab.show()
# plot
pylab.plot(test3.sample_times, test3)
pylab.show()

visualize(test1)
visualize(test3)

'''

'\n# test\ntest1 = burried_gw(30,35,1000)\ntest2 = burried_flip(30,35,1000)\ntest3 = burried_false(-21,1000)\n\n# plot\npylab.plot(test1.sample_times, test1)\npylab.show()\n# plot\npylab.plot(test2.sample_times, test2)\npylab.show()\n# plot\npylab.plot(test3.sample_times, test3)\npylab.show()\n\nvisualize(test1)\nvisualize(test3)\n\n'

In [172]:
# define a function to transform numpy array to pycbc timeseries.

def convert(w):
    # get sample times
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=20,    
                          mass2=20,
                          delta_t=1.0/4096,
                          f_lower=20,
                          distance=100)
    hp.resize(4096)
    hp.start_time = 0
    times = hp.sample_times
    
    wform = pd.Series(w, index =times)
    
    # get pycbc series!
    f_path = 'temporary.txt'
    f= open(f_path,"w+").close()
    wform.to_csv(f_path,sep=' ')
    wform = load_timeseries(f_path)
    
    return wform

In [173]:
# Output one True after every k False, total length n
# For data selection purpose
def bol(k,n):
    bol = []
    for i in np.arange(n/(k)):
        for l in np.arange(k-1):
            bol.append(False)
        bol.append(True)
    return bol

In [174]:
# return partial_burried_false_signals 
# magnitude and snr same as burried_false
# percentage: keep 0 to 1 of the waveform, randomly chosen from anypart of the original waveform

def partial_burried_false(mag,snr,percentage):
    fal = random_false_sig(mag)
    fal.resize(4096)
    noi = noise(fal,snr)
    
    # left + right = 1 - percentage
    left = uniform(0,1-percentage)
    right = (1-percentage)-left
    fal = fal.crop(left, right)
    
    # complete to 4096 data
    zeros = 4096 - len(fal)
    left_zeros = randint(zeros+1)
    right_zeros = zeros - left_zeros 
    fal.append_zeros(right_zeros)
    fal.prepend_zeros(left_zeros)
    
    fal.start_time = 0
    
    return fal+noi
    

In [175]:
# return partial_burried_signals 
# magnitude and snr same as burried_false
# percentage: keep 0 to 1 of the waveform, randomly chosen from anypart of the original waveform


def partial_burried_gw(m1,m2,snr,percentage):

    #differ from false signals because not the whole 1 second is gw
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=m1,    
                          mass2=m2,
                          delta_t=1.0/4096,
                          f_lower=20,
                          distance=100)
    
    # crop the zeros that usually follows the merger
    # length of the merger signal (up till 0.0s the merge and add 0.05 for the remaining impact after 1 second)
    sigtime = - hp.start_time + 0.05
    totaltime = hp.duration
    hp.start_time = 0
    hp = hp.crop(0, totaltime - sigtime)
    
    # left + right = 1 - percentage
    left = uniform(0, 1-percentage) * sigtime
    right = ((1-percentage)-left) * sigtime
    hp = hp.crop(left, right)
    
    # complete to 4096 data
    zeros = 4096 - len(hp)
    left_zeros = randint(zeros+1)
    right_zeros = zeros - left_zeros 
    hp.append_zeros(right_zeros)
    hp.prepend_zeros(left_zeros)
    
    noi = noise(hp,snr)
    
    hp.start_time = 0
    
    return noi + hp

In [176]:
# stronger version of last one: open up mass range to be anything, because we can just crop it to 1 second
def partial_burried_gw_jr(m1,m2,snr,percentage):
       
    #differ from false signals because not the whole 1 second is gw
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=m1,    
                          mass2=m2,
                          delta_t=1.0/4096,
                          f_lower=20,
                          distance=100)
    noi = noise(hp,snr)
    
    sigtime = - hp.start_time + 0.05
    totaltime = hp.duration
    hp.start_time = 0
    hp = hp.crop(0, totaltime - sigtime)
    
    # left + right = 1 - percentage
    left = uniform(0, 1-percentage) * sigtime
    right = ((1-percentage)-left) * sigtime
    hp = hp.crop(left, right)
    
    if hp.duration <= 1:
        # complete to 4096 data
        zeros = 4096 - len(hp)
        left_zeros = randint(zeros+1)
        right_zeros = zeros - left_zeros 
        hp.append_zeros(right_zeros)
        hp.prepend_zeros(left_zeros)
    
    else:
        # alarm message 
        print("Input percentage does not match actual returned signal percentage because waveform is too long. The beginning is thus cropped."   )
        hp = hp.crop(hp.duration-1, 0)
    
    noi = noise(hp,snr)    
    hp.start_time = 0
        
    return noi + hp

Partially burried flipped gw maybe too hard to be distinguished from non-flipped ones so did not include.

One concern is that if it happens so the crop cropped out the merger range, is it still possible to be recognized by the machine.

Notice for high masses the merger signal really short only 0.1 so should be careful about cropping.

The case for waveform longer than 1 sec is included in the junior version.


In [159]:
'''
# test the functions

A  = partial_burried_false(-21,600,0.3)
pylab.plot(A.sample_times, A)
pylab.show()

B  = partial_burried_gw(50,70,600,0.7)
pylab.plot(B.sample_times, B)
pylab.show()

C  = partial_burried_gw_jr(50,70,600,0.7)
pylab.plot(C.sample_times, C)
pylab.show()

C  = partial_burried_gw_jr(15,30,600,0.7)
pylab.plot(C.sample_times, C)
pylab.show()
'''

'\n# test the functions\n\nA  = partial_burried_false(-21,600,0.3)\npylab.plot(A.sample_times, A)\npylab.show()\n\nB  = partial_burried_gw(50,70,600,0.7)\npylab.plot(B.sample_times, B)\npylab.show()\n\nC  = partial_burried_gw_jr(50,70,600,0.7)\npylab.plot(C.sample_times, C)\npylab.show()\n\nC  = partial_burried_gw_jr(15,30,600,0.7)\npylab.plot(C.sample_times, C)\npylab.show()\n'

# For 2B False signals that resemble gw

In [181]:
# return partially burried gausspulse signals as coded for 2B
# percentage: keep 0 to 1 of the waveform, randomly chosen from anypart of the original waveform

def partial_burried_2B(wf,snr,percentage):
    fal = wf
    fal.resize(4096)
    noi = noise(fal,snr)
    
    # left + right = 1 - percentage
    left = uniform(0,1-percentage)
    right = (1-percentage)-left
    fal = fal.crop(left, right)
    
    # complete to 4096 data
    zeros = 4096 - len(fal)
    left_zeros = randint(zeros+1)
    right_zeros = zeros - left_zeros 
    fal.append_zeros(right_zeros)
    fal.prepend_zeros(left_zeros)
    
    fal.start_time = 0
    
    return fal+noi
    